In [ ]:
from avapi.carla import CarlaScenesManager
from avapi.nuscenes import nuScenesManager
from avapi.ugv import UgvScenesManager


carla_data_root = "/data/shared/CARLA/multi-agent-random"
nuscenes_data_root = "/data/shared/nuScenes"
ugv_data_root = "/data/shared/ugv/WILK_BASEMENT"

SMs = [
    ("lidar-0", CarlaScenesManager(data_dir=carla_data_root)),
    ("LIDAR_TOP", nuScenesManager(data_dir=nuscenes_data_root, split="v1.0-trainval")),
    ("lidar", UgvScenesManager(data_dir=ugv_data_root)),
]

For dataset CARLA:

TRAIN -- 10 agent, 5 scenes, 5005 frames, 10000 point clouds, 10.0 agents, 6.4555 objects per frame per agent 

For dataset CARLA:

VAL -- 10 agent, 4 scenes, 4004 frames, 8000 point clouds, 10.0 agents, 5.110375 objects per frame per agent

For dataset CARLA:

TEST -- 10 agent, 1 scenes, 1001 frames, 2000 point clouds, 10.0 agents, 7.145 objects per frame per agent 

nuscenes

TRAIN -- 1 agents, 700 scenes, 27533 frames, 27533 point clouds, 0.9785714285714285 agents, 24.98634366033487 objects per frame per agent 

For dataset nuScenes:

VAL -- 1 agents, 150 scenes, 6019 frames, 6019 point clouds, 1.0 agents, 22.944342914105334 objects per frame per agent 

In [ ]:
for lid_sensor, SM in SMs[2:]:
    for split in ["train", "val", "test"]:
        n_scenes = len(SM.splits_scenes[split])
        n_frames = n_pcs = n_agents = n_objs = 0
        for scene in SM.splits_scenes[split]:
            try:
                DM = SM.get_scene_dataset_by_name(scene)
            except Exception as e:
                print("Exception trying this datasets...continuing")
                continue
            n_frames += len(DM.frames)
            agent_set =  DM.get_agent_set(DM.frames[0])
            for agent_ID in agent_set:
                n_agents += 1
                frames_pc = DM.get_frames(sensor=lid_sensor, agent=agent_ID)
                n_pcs += len(frames_pc)
                for frame in frames_pc:
                    try:
                        n_objs += len(
                            DM.get_objects(frame=frame, sensor=lid_sensor, agent=agent_ID)
                        )
                    except FileNotFoundError:
                        pass

        # print out results
        if n_scenes > 0:
            print(
                f"For dataset {SM.name}:\n"
                f"{split.upper()} -- {len(agent_set)} agents, {n_scenes} scenes, {n_frames} frames, {n_pcs} "
                f"point clouds, {n_agents/n_scenes} agents, {n_objs / n_pcs} objects per frame per agent "
            )
        else:
            print(
                f"For dataset {SM.name}:\n {split.upper()} has no scenes"
            )